# Load and Prepare Raw data and Save tables


In [2]:
from typing import List
import requests
import json
import pandas as pd
import numpy as np
import time
import sys
from collections.abc import MutableMapping,Iterable
import datetime


def get_API_response(url):
    response_API = requests.get(url)
    #print('Status Code-', response_API.status_code)
    #print('Loading Data--')
    start = time.time()
    raw_data = response_API.text
    # Parse Json
    data = json.loads(raw_data)
    #print('Time taken to load data-',round((time.time()-start),3))
    return data
    
def extract_dict_keys(dict_int, select_keys):
     dict_out = {key:value for key,value in dict_int.items() if key in select_keys}
     return dict_out

R_ExtractDate = datetime.datetime.today().strftime("%Y-%m-%d")
frax_start_date= '2020-12-21'

# Stable Coin - Protocol map info

In [47]:
# Load manual SC- Protocol information 
pdf_SC_protocol_info = pd.read_csv('../data_checks/SC_protocol_info_manual.csv')

In [48]:
pdf_SC_protocol_info.head(5)

,id,name,symbol,gecko_id,priceSource,price,Protocol,Symbol,defilama_present,Comments
0,10,Magic Internet Money,MIM,magic-internet-money,chainlink,0.997343,AbracadabrA,SPELL,True,NaN
1,45,Acala Dollar,AUSD,acala-dollar,NaN,NaN,Acala (ACA),ACA,True,Multiple Protocols
2,20,Alchemix USD,ALUSD,alchemix-usd,curve,0.987148,Alchemix,ALCX,True,NaN
3,38,STBL,STBL,stbl,NaN,NaN,Algofi,ALGO,True,NaN
4,48,BAI Stablecoin,BAI,bai-stablecoin,dexscreener,0.996700,AstridDAO,ATID,True,NaN


In [4]:
## TVL related data (defilama)
# Get All protocols in defilama along with their tvl
url_tvl_protocols = 'https://api.llama.fi/protocols'
raw_tvl_protocols = get_API_response(url_tvl_protocols) 
raw_tvl_protocols_list = raw_tvl_protocols
pdf_tvl_protocols_allcols = pd.json_normalize(raw_tvl_protocols_list, sep='_')
# Select important coplumns
select_columns = ['id', 'name', 'address', 'symbol','chain', 'gecko_id',\
     'cmcId','category', 'chains' ,'oracles', 'slug','tvl','fdv', 'mcap']
    
pdf_tvl_protocols = pdf_tvl_protocols_allcols[select_columns]
# add extract date as the values are based in current date
pdf_tvl_protocols['R_ExtractDate'] = R_ExtractDate
# 1738 protocols enlisted

<ipython-input-4-b1a476281e81>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_tvl_protocols['R_ExtractDate'] = R_ExtractDate


In [50]:
# Extract Protocol information of stabel coins
prot_names = set(pdf_SC_protocol_info['Protocol'].values)
prot_symbols = set(pdf_SC_protocol_info['Symbol'].values)
coin_symbols = 	set(pdf_SC_protocol_info['symbol'].values)

pdf_SC_protocols_map = pdf_tvl_protocols[pdf_tvl_protocols['name'].isin(prot_names)
                                     | pdf_tvl_protocols['symbol'].isin(prot_symbols)
                                     | pdf_tvl_protocols['symbol'].isin(coin_symbols)]

In [45]:
pdf_SC_protocols_map.sort_values("symbol",axis=0)

,id,name,address,symbol,chain,gecko_id,audits,cmcId,category,chains,oracles,slug,tvl,fdv,mcap,R_ExtractDate
121,1087,Algofi,None,-,Algorand,None,2,None,Lending,[Algorand],[TWAP],algofi,8.775495e+07,NaN,NaN,2022-08-03
70,1879,Acala LCDOT,acala:-,ACA,Acala,None,2,None,Liquid Staking,[Acala],[],acala-lcdot,1.945281e+08,NaN,NaN,2022-08-03
149,1849,Acala Liquid-Staking,acala:-,ACA,Acala,None,2,None,Liquid Staking,[Acala],[],acala-liquid-staking,5.809474e+07,NaN,NaN,2022-08-03
355,1847,Acala Swap,acala:-,ACA,Acala,None,2,None,Dexes,[Acala],[],acala-swap,9.494883e+06,NaN,NaN,2022-08-03
86,204,Alchemix,0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF,ALCX,Multi-Chain,alchemix,2,8613,Synthetics,"[Ethereum, Fantom]",[],alchemix,1.327473e+08,8.436790e+07,52652209.0,2022-08-03
371,1708,AstridDAO,astar:0x5271D85CE4241b310C0B34b7C2f1f036686A6d7C,ATID,Astar,None,2,None,CDP,[Astar],[DIA],astriddao,8.203670e+06,NaN,NaN,2022-08-03
697,497,Float Protocol,0x24a6a37576377f63f194caa5f518a60f45b42921,BANK,Ethereum,float-protocol,2,8612,Yield,[Ethereum],NaN,float-protocol,9.316643e+05,9.885220e+05,790303.0,2022-08-03
126,864,Wrapped BNB,kava:-,BNB,Binance,None,0,None,Bridge,[Binance],NaN,wrapped-bnb,8.211795e+07,NaN,NaN,2022-08-03
1289,1387,DEUS Finance,0xde5ed76e7c05ec5e4572cfc88d1acea165109e44,DEUS,Multi-Chain,deus-finance-2,0,12972,CDP,"[Ethereum, Polygon, Fantom]",[],deus-finance,7.066664e+03,1.000019e+07,3393135.0,2022-08-03
196,123,dForce,0x431ad2ff6a9c365805ebad47ee021148d6f7dbe0,DF,Multi-Chain,dforce-token,2,4758,Lending,"[Binance, Arbitrum, Ethereum, Polygon, Optimis...","[Chainlink, Band]",dforce,3.586208e+07,4.445458e+07,18940484.0,2022-08-03


In [51]:
# pdf_SC_protocols.shape (41, 16)
# Save dataframe to csv file
# pdf_SC_protocols_map.to_csv('../data_raw/Protocols_stableCoins_map.csv',index=False)

In [88]:
# Manual Search
# SC_protocol_name_same = ["CUSD","USDK","USTC","USN","USDP","USDD"]

pdf_SC_protocols_map[pdf_SC_protocols_map['name'].str.contains("Acala")]
# | pdf_tvl_protocols['symbol'].isin(prot_symbols) 

,id,name,name_hist_tvl,address,symbol,chain,gecko_id,audits,cmcId,category,chains,oracles,slug,tvl,fdv,mcap,R_ExtractDate
10,1879,Acala LCDOT,Acala LCDOT,acala:-,ACA,Acala,NaN,2,NaN,Liquid Staking,['Acala'],[],acala-lcdot,1.945281e+08,NaN,NaN,03-08-2022
18,1849,Acala Liquid-Staking,Acala Liquid-Staking,acala:-,ACA,Acala,NaN,2,NaN,Liquid Staking,['Acala'],[],acala-liquid-staking,5.809474e+07,NaN,NaN,03-08-2022
30,1847,Acala Swap,Acala Swap,acala:-,ACA,Acala,NaN,2,NaN,Dexes,['Acala'],[],acala-swap,9.494883e+06,NaN,NaN,03-08-2022


## historical TVL of SC protocols

In [80]:
pdf_SC_protocols_map = pd.read_csv('../data_raw/Protocols_stableCoins_map.csv')
SC_protocols_map_names = pdf_SC_protocols_map['name_hist_tvl'].values
# len(SC_protocols_map)

41

In [81]:
# Extarct historical-tvl of top 50 common(mcap+tvl) = 78 protocols
hist_tvl_keys = ['id', 'name', 'symbol','tvl', 'tokensInUsd'] # 'misrepresentedTokens'
list_pdf_protocol_hist_tvl = []
for protocol in SC_protocols_map_names:
    print(protocol)
    try:
        url_protocol_hist_tvl = 'https://api.llama.fi/protocol/'+protocol
        raw_protocol_hist_tvl = get_API_response(url_protocol_hist_tvl) 
        
        raw_protocol_hist_tvl_select = extract_dict_keys(raw_protocol_hist_tvl, hist_tvl_keys) 
        id = raw_protocol_hist_tvl_select['id']
        name = raw_protocol_hist_tvl_select['name']
        symbol = raw_protocol_hist_tvl_select['symbol']
       
        protocol_hist_tvl_list = raw_protocol_hist_tvl_select['tvl']
        pdf_protocol_hist_tvl = pd.json_normalize(protocol_hist_tvl_list, sep='_')
        pdf_protocol_hist_tvl['date'] = pd.to_datetime(pdf_protocol_hist_tvl['date'] , unit='s')
        print(f"starting from - {min(pdf_protocol_hist_tvl['date'])}")
        pdf_protocol_hist_tvl = pdf_protocol_hist_tvl[pdf_protocol_hist_tvl['date']>= frax_start_date]
        pdf_protocol_hist_tvl['id'] = id
        pdf_protocol_hist_tvl['name'] = name
        pdf_protocol_hist_tvl['symbol'] = symbol

        list_pdf_protocol_hist_tvl.append(pdf_protocol_hist_tvl)
    except Exception as e:
        print('Not present and skipping..',protocol)
        
pdf_protocol_hist_tvl = pd.concat(list_pdf_protocol_hist_tvl)
pdf_protocol_hist_tvl.reset_index(inplace=True)
pdf_protocol_hist_tvl.drop('index',axis=1, inplace = True)
pdf_protocol_hist_tvl['R_ExtractDate'] = R_ExtractDate

MakerDAO
starting from - 2019-01-04 00:00:00
JustLend
starting from - 2021-11-01 00:00:00
JustCryptos
starting from - 2021-10-21 00:00:00
Frax
starting from - 2020-12-17 00:00:00
JustStables
starting from - 2020-04-03 00:00:00
Venus
starting from - 2020-11-24 00:00:00
Synthetix
starting from - 2019-08-09 22:00:00
Liquity
starting from - 2021-04-06 00:00:00
Abracadabra
starting from - 2021-06-05 00:00:00
benqi
Not present and skipping.. benqi
Acala LCDOT
Not present and skipping.. Acala LCDOT
Alchemix
starting from - 2021-11-18 00:00:00
Neutrino
starting from - 2021-07-07 00:00:00
Algofi
starting from - 2021-12-17 00:00:00
yeti-finance
starting from - 2022-04-19 00:00:00
Wrapped-BNB
starting from - 2021-11-22 00:00:00
QiDao
starting from - 2021-05-02 23:00:00
fei-protocol
starting from - 2021-04-04 00:00:00
Acala Liquid-Staking
Not present and skipping.. Acala Liquid-Staking
origin-dollar
starting from - 2021-04-28 05:00:00
mStable
starting from - 2020-10-29 23:00:00
MoneyOnChain
starti

In [84]:
pdf_protocol_hist_tvl = pdf_protocol_hist_tvl[["id","name","symbol","date","totalLiquidityUSD","R_ExtractDate"]]


,id,name,symbol,date,totalLiquidityUSD,R_ExtractDate
0,118,MakerDAO,MKR,2020-12-21 00:00:00,2.569296e+09,2022-08-03
1,118,MakerDAO,MKR,2020-12-22 00:00:00,2.446120e+09,2022-08-03
2,118,MakerDAO,MKR,2020-12-23 00:00:00,2.463407e+09,2022-08-03
3,118,MakerDAO,MKR,2020-12-24 00:00:00,2.441394e+09,2022-08-03
4,118,MakerDAO,MKR,2020-12-25 00:00:00,2.360728e+09,2022-08-03
...,...,...,...,...,...,...
13253,1717,CoinFLEX,FLEX,2022-07-30 00:00:00,0.000000e+00,2022-08-03
13254,1717,CoinFLEX,FLEX,2022-07-31 00:00:00,0.000000e+00,2022-08-03
13255,1717,CoinFLEX,FLEX,2022-08-01 00:00:00,0.000000e+00,2022-08-03
13256,1717,CoinFLEX,FLEX,2022-08-02 00:00:00,0.000000e+00,2022-08-03


In [85]:
# pdf_protocol_hist_tvl.to_csv('../data_raw/Protocols_stableCoins_map_hist_tvl.csv',index=False)